### ВХОДНЫЕ ДАННЫЕ:

In [1]:
import numpy as np

                                            #ВХОДНЫЕ ДАННЫЕ
m = 4 #Размерность входного массива(ДхШ)
k = 2 #Размерность ядра свертки (ДхШ)
m_k_1 = (m-k)+1 #Размерность одной карты скрытого слоя свертки (ДхШ)
stok_w = 2 #Число ядер свертки и соответственно количество карт срытого слоя (ДхШ)
stob_w = k*k #Количество элементов в ядре свертки между входным и скрытым слоями

x1 = np.arange(1,m*m+1).reshape(m,m) #Входной 2D массив данных
w1 = np.arange(1,stok_w*stob_w+1).reshape(stok_w, k,k) #Ядра свертки (веса)
x2 = np.zeros((stok_w, m_k_1, m_k_1)) #Выходной массив (карты скрытого слоя)
#(s1, f1, f1) = x1.shape # Получаем размеры выходного массива
(s2, f2, f2) = x2.shape # Получаем размеры выходного массива 
e2 = np.arange(1,s2*f2*f2+1).reshape(stok_w, m_k_1, m_k_1) #Ошибка скрытого слоя (размеры должны совпадать с картой этого слоя)
lr = 0.01 #Скорость обучения 
print ('x1: ','\n', x1, '\n')
print ('w1: ','\n',w1, '\n')

x1:  
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]] 

w1:  
 [[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]] 



### СВЁРТКА:

In [2]:
                                            # ОПЕРАЦИЯ СВЁРТКИ
print('Вход x1:\n', x1)
print('Вес w1:\n',w1)

for s in range(stok_w): # Цикл по количеству ядер свёртки
    for h in range(m_k_1): # Цикл по количеству проходов ядра свёртки, по горизонтали от входных данных
        for w in range(m_k_1): # Цикл по количеству проходов ядра свёртки, по вертикали от входных данных
            x2 [s,h,w] = np.sum(x1[h:h+k, w:w+k] * w1[s]) # Сумма поэлементного умножения области входных данных с ядром

print('Сверточный слой x2:\n', x2, '\n')

Вход x1:
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
Вес w1:
 [[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]
Сверточный слой x2:
 [[[  44.   54.   64.]
  [  84.   94.  104.]
  [ 124.  134.  144.]]

 [[ 100.  126.  152.]
  [ 204.  230.  256.]
  [ 308.  334.  360.]]] 



### ОБНОВЛЕНИЕ ВЕСОВ ЯДРА СВЁРТКИ:

In [3]:
print('Вход x1:\n', x1)
print('Ошибка e2:\n', e2)

def Xrot_180(X_rot_180):
    #for s in range(stok_w):
    X_rot_180 = np.fliplr(X_rot_180)
    X_rot_180 = np.flipud(X_rot_180)
    return X_rot_180

for s in range(stok_w): # Цикл по количеству ядер свёртки
    #for h in range(m-(m_k_1)+1): 
    for h in range(k): # Цикл по количеству элементов ядра свёртки, по горизонтали
        #for w in range(m-(m_k_1)+1): 
        for w in range(k): # Цикл по количеству элементов ядра свёртки, по вертикали
            w1[s, h, w] = np.sum(e2[s] * Xrot_180(x1[h:h+m_k_1, w:w+m_k_1])) # Сумма поэлементного умножения области входных данных 
            #и ошибки скрытого слоя

print('Обновленные веса w1:\n', w1)

Вход x1:
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
Ошибка e2:
 [[[ 1  2  3]
  [ 4  5  6]
  [ 7  8  9]]

 [[10 11 12]
  [13 14 15]
  [16 17 18]]]
Обновленные веса w1:
 [[[ 192  237]
  [ 372  417]]

 [[ 678  804]
  [1182 1308]]]


### ОБРАТНОЕ РАСПРОСТРОНЕНИЕ ОШИБКИ:

In [4]:
                    # ОБРАТНОЕ РАСПРОСТРОНЕНИЕ ОШИБКИ
# Функция переворота матрицы весов на 180
e1 = np.zeros((stok_w, m, m)) #Ошибка входного слоя
def rot_180(W_rot_180):
    for s in range(stok_w):
        W_rot_180[s] = np.fliplr(W_rot_180[s])
        W_rot_180[s] = np.flipud(W_rot_180[s])
    return W_rot_180

rot180_w1 = rot_180(w1)
print('Перевернутые веса rot180_w1 :\n', rot180_w1)

# Создадим матрицу размера - для прогона весов как показано на слайде
e2_temp = np.zeros((stok_w, m+k-1, m+k-1))
# Поместим в центр ошибку на предыдущем слое
for s in range(stok_w):
    e2_temp[s, k-1:m_k_1+k-1, k-1:m_k_1+k-1] = e2[s]    

print('Матрица размера - для прогона весов как показано на слайде e2_temp:\n', e2_temp)

# Проходим по этой матрице перевернутыми ядрами
for s in range(stok_w):
    for h in range(m):
        for w in range(m):
            #e1[s,h,w] = np.sum(lr * e2_temp[s, h:h+k, w:w+k] * rot180_w1[s])
            e1[s,h,w] = np.sum(lr * e2_temp[s, h:h+k, w:w+k] * rot180_w1[s] * 1/1+np.exp(-x1[h, w]) * (1 - 1/1+np.exp(-x1[h, w])))
print('Ошибка до суммы - e1:\n', e1)

# Cумма ошибки (значения храним в 1м массиве)
for s in range(stok_w-1):
    e1[0] = e1[s] + e1[s+1]
print('Ошибка после суммы - e1:\n', e1)
print('Ошибка входного слоя - e1:\n', e1[0])

Перевернутые веса rot180_w1 :
 [[[ 417  372]
  [ 237  192]]

 [[1308 1182]
  [ 804  678]]]
Матрица размера - для прогона весов как показано на слайде e2_temp:
 [[[  0.   0.   0.   0.   0.]
  [  0.   1.   2.   3.   0.]
  [  0.   4.   5.   6.   0.]
  [  0.   7.   8.   9.   0.]
  [  0.   0.   0.   0.   0.]]

 [[  0.   0.   0.   0.   0.]
  [  0.  10.  11.  12.   0.]
  [  0.  13.  14.  15.   0.]
  [  0.  16.  17.  18.   0.]
  [  0.   0.   0.   0.   0.]]]
Ошибка до суммы - e1:
 [[[   2.46134113    6.28326256   10.50991501    7.11134185]
  [  11.4001816    30.69002458   42.87000333   26.73000045]
  [  28.32000006   67.23000001   79.41         46.35      ]
  [  26.04         58.95         66.84         37.53      ]]

 [[  68.34134113  155.05326256  169.80991501   96.48134185]
  [ 206.3401816   460.26002458  499.98000333  277.56000045]
  [ 262.14000006  579.42000001  619.14        340.92      ]
  [ 189.12        410.22        435.12        235.44      ]]]
Ошибка после суммы - e1:
 [[[  70.80268

### МАКСПУЛ:

In [5]:
print('ПРОГОН ВПЕРЕД:\n ')
import re # Для извлечения дробной и целой части

x22 = np.arange(stok_w*m*m).reshape(stok_w,m,m) #выходной слой
print('Скрытый слой x22:\n', x22)

# Пулинг данные
pool_m = 2 # Размер матрицы ядра пулинга (ДхШ)
m_k_1_pool = int(m/pool_m) # Размерность слоя пулинга (целая часть)
x22_maxpool = np.zeros((stok_w, m_k_1_pool, m_k_1_pool)) # Матрица выходного массива пулинга
x22_el = np.zeros((stok_w, m_k_1_pool, m_k_1_pool), dtype='<U32') # хранит адрес макс элемента

# Операция пулинга
for s in range(stok_w):
    for h in range(m_k_1_pool):
        for w in range(m_k_1_pool):
            temp2 = x22[s, h*pool_m:h*pool_m+pool_m, w*pool_m:w*pool_m+pool_m] # Для хранения подматрицы с нулевыми и макс знач.
            x22_maxpool[s,h,w] = x22[s, h*pool_m:h*pool_m+pool_m, w*pool_m:w*pool_m+pool_m].max() # Матрица выходного массива после пулинга
            # Здесь же, в циклах пулинга запоминаем координаты максимальных элементов в своих областях 
            for i in range(pool_m):
                for j in range(pool_m):
                    if temp2[i, j] == x22_maxpool[s,h,w]:
                        temp3 = str(i+h*pool_m) + ',' + str(j+w*pool_m) #Запоминаем,через запятую, координаты по строкам и столбцам
                        
            x22_el[s, h, w] = temp3 
            
print('x22_maxpool слой:\n ',x22_maxpool)
print('Адрес (индекс) максимального элемента x22_el:\n ',x22_el)

print('\n\nПРОГОН НАЗАД:\n ')
#После вычисления ошибки в слое пулинга, распростроняем эту ошибку на слой перед пулингом согласно индексам, остальные нули
# Обнулим ошибки перед пулингом
e1 = np.zeros((stok_w, m, m))
# Запишем сюда ошибки на пулинге согластно индексам x22_el
x22e_maxpool = x22_maxpool # Ошибка на слое пулинга
print('Ошибка на слое пуллинга x22e_maxpool:\n ',x22e_maxpool)
for s in range(stok_w):
    for h in range(m_k_1_pool):
        for w in range(m_k_1_pool):
            result = re.split(r'[,]', x22_el[s,h,w]) # Разбивает строку по разделителю ',' можно сразу несколько разделителей 
            i = int(result[0])
            j = int(result[1])
            e1[s,i,j] = x22e_maxpool[s,h,w]
print('Ошибка на слое перед пуллингом e1:\n ',e1)

ПРОГОН ВПЕРЕД:
 
Скрытый слой x22:
 [[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]
  [12 13 14 15]]

 [[16 17 18 19]
  [20 21 22 23]
  [24 25 26 27]
  [28 29 30 31]]]
x22_maxpool слой:
  [[[  5.   7.]
  [ 13.  15.]]

 [[ 21.  23.]
  [ 29.  31.]]]
Адрес (индекс) максимального элемента x22_el:
  [[['1,1' '1,3']
  ['3,1' '3,3']]

 [['1,1' '1,3']
  ['3,1' '3,3']]]


ПРОГОН НАЗАД:
 
Ошибка на слое пуллинга x22e_maxpool:
  [[[  5.   7.]
  [ 13.  15.]]

 [[ 21.  23.]
  [ 29.  31.]]]
Ошибка на слое перед пуллингом e1:
  [[[  0.   0.   0.   0.]
  [  0.   5.   0.   7.]
  [  0.   0.   0.   0.]
  [  0.  13.   0.  15.]]

 [[  0.   0.   0.   0.]
  [  0.  21.   0.  23.]
  [  0.   0.   0.   0.]
  [  0.  29.   0.  31.]]]
